In [30]:
import os
import torch
import dataflow as dataflow
import model as models
import utils as utils

import lightning as L
from omegaconf import OmegaConf
from lightning.pytorch.callbacks import DeviceStatsMonitor
import time
import wandb
import argparse

import multiprocessing
import psutil


cfg_path = '../configs/local-transformer-backbone.yaml'
cfg = OmegaConf.load(cfg_path)

In [31]:
cfg.hyperparameters.num_workers = 0
cfg.datafiles.data_dir = '../data/training.1600000.processed.noemoticon.csv'

dm = dataflow.TweetDataModule(cfg)
dm.prepare_data()

# splits/transforms
dm.setup(stage="fit")

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [32]:
sample = next(iter(dm.train_dataloader()))

In [33]:
sample

({'input_ids': tensor([[57344,    32,   119,  ...,     0,     0,     0],
          [57344,    87,    97,  ...,     0,     0,     0],
          [57344,    79,    75,  ...,   121,    32, 57345],
          ...,
          [57344,    67,    72,  ...,     0,     0,     0],
          [57344,    32,   115,  ...,     0,     0,     0],
          [57344,    32,    97,  ...,     0,     0,     0]]),
  'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0]])},
 tensor([1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
         0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
         1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0]))

In [34]:
import transformer
t = transformer.TransformerEncoder(input_size=dm.tokenizer.vocab_size, 
                                   hidden_size=32, num_layers=4, num_heads=4, dropout=0.1)

In [35]:
sample[0]['input_ids'].shape  # B, seq_len

torch.Size([64, 140])

In [36]:
t(sample[0]['input_ids']).shape

torch.Size([64, 2])

In [1]:
import pandas as pd
import numpy as np

In [2]:

DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
df = pd.read_csv(
    'data/Sentiment140-dataset/training.1600000.processed.noemoticon.csv',
    encoding=DATASET_ENCODING,
    names=DATASET_COLUMNS
    )

In [3]:
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
print("Dataset size:", len(df))

Dataset size: 1600000


In [5]:
df['target'].unique()

array([0, 4], dtype=int64)

### Preprocessing

In [6]:
decode_map = {0: 0, 4: 1}
def binarize_sentiment(label):
    return decode_map[int(label)]

df['target'] = df.target.apply(lambda x: binarize_sentiment(x))
df['target'].unique()

array([0, 1], dtype=int64)

In [7]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, train_size=1000, test_size=100, random_state=42, stratify=df['target'])
train_df, val_df = train_test_split(train_df, test_size=0.3, random_state=42, stratify=train_df['target'])

train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)
test_df.reset_index(inplace=True, drop=True)

In [9]:
import importlib
from torch.utils.data import DataLoader
import src.dataflow as dataflow
import src.model as models

import pytorch_lightning as L

In [17]:

importlib.reload(dataflow)
importlib.reload(models)

from transformers import CanineTokenizer

# single sequence: [CLS] X [SEP]
tokenizer = CanineTokenizer.from_pretrained("google/canine-c", 
                                            return_tensors='pt')

# Build Datasets
train_dataset = dataflow.TweetDataset(train_df['text'], train_df['target'], tokenizer)
val_dataset = dataflow.TweetDataset(val_df['text'], val_df['target'], tokenizer)
test_dataset = dataflow.TweetDataset(test_df['text'], test_df['target'], tokenizer)


# Build DataLoaders
BATCH_SIZE = 4
SHUFFLE = False
train_dataloader = DataLoader(train_dataset, 
                              batch_size=BATCH_SIZE, 
                              shuffle=SHUFFLE, 
                              collate_fn=lambda b: dataflow.collate_fn(b, input_pad_token_id=tokenizer.pad_token_id)
                              )
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE, collate_fn=lambda b: dataflow.collate_fn(b, input_pad_token_id=tokenizer.pad_token_id))
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE, collate_fn=lambda b: dataflow.collate_fn(b, input_pad_token_id=tokenizer.pad_token_id))

sample, labels = next(iter(train_dataloader))
print(sample)
print(labels)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


{'input_ids': tensor([[57344,    32,   100,   105,   100,    32,   121,   111,   117,    32,
           108,   105,   107,   101,    32,    98,   101,   105,   110,   103,
            32,   109,   111,    98,    98,   101,   100,    32,    98,   121,
            32,   116,   104,    97,   116,    32,   103,   114,   111,   117,
           112,    32,   111,   102,    32,   103,   105,   114,   108,   115,
            32,    32,   108,   111,   108, 57345,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [57344,    32,   110,   111,   119,    32,    99,    97,   115,   104,
            32,   117,    32,   107,   110,   111,    32,    73,    39,   109,
            32,   110,   111,   116,    32,    97,    32,   104,    97,   116,
           101,   114,    46,    46,    46,    46,    46,   106,   117,   115,
            32,   117,   112,   115,   101,   116,    32,    73,    32, 

In [87]:
sample['input_ids'].shape

torch.Size([4, 75])

In [97]:
importlib.reload(dataflow)
importlib.reload(models)
from pytorch_lightning.callbacks import EarlyStopping

print(f"Training on {len(train_df)} examples...")
NUM_EPOCHS = 4

early_stop = EarlyStopping('val_loss', patience=5, verbose=True)

trainer = L.Trainer(max_epochs=NUM_EPOCHS, callbacks=[ early_stop ])
classifier = models.SentimentClassifier(tokenizer=tokenizer, freeze_encoder=True)
trainer.fit(classifier, train_dataloader, val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Training on 700 examples...



  | Name            | Type             | Params
-----------------------------------------------------
0 | encoder         | CanineModel      | 132 M 
1 | dropout         | Dropout          | 0     
2 | classifier_head | Sequential       | 592 K 
3 | criterion       | CrossEntropyLoss | 0     
-----------------------------------------------------
592 K     Trainable params
132 M     Non-trainable params
132 M     Total params
530.700   Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\boyla\miniconda3\envs\sentiment\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


passed encoder: CanineModelOutputWithPooling(last_hidden_state=tensor([[[ 6.7530e-02,  2.5443e-01,  8.1614e-02,  ...,  3.2349e-01,
           4.5179e-01,  1.5472e-03],
         [ 4.5456e-01,  3.4737e-01, -2.2940e-01,  ..., -3.3214e-01,
           3.1136e-01,  7.3494e-03],
         [ 1.9320e-02,  2.8060e-01,  3.6783e-01,  ..., -6.2053e-03,
           8.0185e-01, -1.7725e-01],
         ...,
         [-1.9998e-01, -2.2607e-01,  4.4475e-01,  ...,  3.4933e-01,
           5.9028e-01, -1.2415e-01],
         [-1.8726e-01, -2.3769e-01,  4.4339e-01,  ...,  2.9612e-01,
           2.8135e-01, -1.4380e-01],
         [-4.4759e-03,  1.1550e-01,  2.1787e-01,  ...,  5.1942e-01,
           2.0116e-01,  2.5695e-01]],

        [[ 8.0821e-02, -3.0083e-01, -1.9129e-02,  ...,  2.4460e-01,
           3.3204e-01, -2.0822e-01],
         [ 3.4863e-01, -1.5260e-01, -3.6323e-01,  ..., -1.1947e-01,
           9.2758e-02, -4.9332e-02],
         [ 5.7820e-02, -1.3415e-02,  6.2693e-01,  ...,  1.9117e-01,
           9.

c:\Users\boyla\miniconda3\envs\sentiment\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/175 [00:00<?, ?it/s] {'input_ids': tensor([[57344,    32,   100,   105,   100,    32,   121,   111,   117,    32,
           108,   105,   107,   101,    32,    98,   101,   105,   110,   103,
            32,   109,   111,    98,    98,   101,   100,    32,    98,   121,
            32,   116,   104,    97,   116,    32,   103,   114,   111,   117,
           112,    32,   111,   102,    32,   103,   105,   114,   108,   115,
            32,    32,   108,   111,   108, 57345,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [57344,    32,   110,   111,   119,    32,    99,    97,   115,   104,
            32,   117,    32,   107,   110,   111,    32,    73,    39,   109,
            32,   110,   111,   116,    32,    97,    32,   104,    97,   116,
           101,   114,    46,    46,    46,    46,    46,   106,   117,   115,
            32,   117, 

c:\Users\boyla\miniconda3\envs\sentiment\Lib\site-packages\pytorch_lightning\trainer\call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
trainer.test(test_dataloader)

In [22]:
importlib.reload(dataflow)
importlib.reload(models)

# load checkpoint
checkpoint = "./lightning_logs/version_9/checkpoints/epoch=3-step=700.ckpt"
m = models.SentimentClassifier.load_from_checkpoint(checkpoint, tokenizer=tokenizer)

# choose your trained nn.Module
m.eval()

text = "@elonmusk is so empathetic and level-headed!!"
print(f"{text} \n-> Sentiment: {m(text)}")

text = "sheep are dumb"
print(f"{text} \n-> Sentiment: {m(text)}")

@elonmusk is so empathetic and level-headed!! 
-> Sentiment: [0]
sheep are dumb 
-> Sentiment: [0]
